## Handling Missing Values, Data Types, and Duplicates
In this section, we clean the dataset by:
- Removing columns or rows with excessive null values
- Converting date columns to datetime format
- Dropping duplicate records

In [6]:
# Load the dataset
import pandas as pd

df = pd.read_csv("../data/ai_job_dataset.csv")

# Show initial info
print("Initial dataset shape:", df.shape)
df.info()

# Drop columns with excessive missing values if any
threshold = 0.5  # drop columns with more than 50% NaNs
null_ratios = df.isna().mean()
cols_to_drop = null_ratios[null_ratios > threshold].index.tolist()
print("Dropping columns with too many missing values:", cols_to_drop)
df.drop(columns=cols_to_drop, inplace=True)

# Drop rows with any remaining missing values
print("Missing values before dropna:", df.isna().sum().sum())
df.dropna(inplace=True)
print("Missing values after dropna:", df.isna().sum().sum())
print("Dataset shape after removing missing values:", df.shape)

# Convert date columns to datetime
if "posting_date" in df.columns:
    df["posting_date"] = pd.to_datetime(df["posting_date"], errors="coerce")

if "application_deadline" in df.columns:
    df["application_deadline"] = pd.to_datetime(df["application_deadline"], errors="coerce")

# Drop duplicate rows
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"Removed {before - after} duplicate rows.")

Initial dataset shape: (15000, 19)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   job_id                  15000 non-null  object 
 1   job_title               15000 non-null  object 
 2   salary_usd              15000 non-null  int64  
 3   salary_currency         15000 non-null  object 
 4   experience_level        15000 non-null  object 
 5   employment_type         15000 non-null  object 
 6   company_location        15000 non-null  object 
 7   company_size            15000 non-null  object 
 8   employee_residence      15000 non-null  object 
 9   remote_ratio            15000 non-null  int64  
 10  required_skills         15000 non-null  object 
 11  education_required      15000 non-null  object 
 12  years_experience        15000 non-null  int64  
 13  industry                15000 non-null  object 
 14  pos

## Encoding Categorical Variables
We transform categorical variables into numerical format to prepare the dataset for regression models. 
For simplicity, we use:
- One-hot encoding for nominal features (e.g., country, company size)
- Label encoding if only a few unique values are present

In [7]:
# Preview categorical columns
cat_cols = df.select_dtypes(include='object').columns.tolist()
print("Categorical columns:", cat_cols)
print(df[cat_cols].nunique())

# Apply one-hot encoding for categorical features with low to medium cardinality
# (we avoid high-cardinality features for now to prevent dimensionality explosion)

max_unique = 30  # adjustable threshold
ohe_cols = [col for col in cat_cols if df[col].nunique() <= max_unique]

df_encoded = pd.get_dummies(df, columns=ohe_cols, drop_first=True)

print("Dataset shape after one-hot encoding:", df_encoded.shape)

Categorical columns: ['job_id', 'job_title', 'salary_currency', 'experience_level', 'employment_type', 'company_location', 'company_size', 'employee_residence', 'required_skills', 'education_required', 'industry', 'company_name']
job_id                15000
job_title                20
salary_currency           3
experience_level          4
employment_type           4
company_location         20
company_size              3
employee_residence       20
required_skills       13663
education_required        4
industry                 15
company_name             16
dtype: int64
Dataset shape after one-hot encoding: (15000, 108)


## Feature Scaling
To ensure that all numerical features contribute equally to the model, we scale them using StandardScaler.
This is especially important for linear models and PCA-based techniques.

In [8]:
from sklearn.preprocessing import StandardScaler

# Select only numeric columns (excluding target if already chosen)
numeric_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns.tolist()

# OPTIONAL: remove target column from scaling
# numeric_cols.remove('your_target_column')  # Uncomment and replace if needed

scaler = StandardScaler()
df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])

print("Feature scaling applied.")

Feature scaling applied.
